In [13]:
# HIDDEN
# This useful nonsense should just go at the top of your notebook.
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plots
import numpy as np
plots.style.use('fivethirtyeight')
# datascience version number of last run of this notebook
version.__version__

'0.5.17'

<h1>Class 10: World War II Veterans, Earnings, Ability, and Health</h1>

Angrist and Krueger (1994) reexamined data on the earnings of veterans of World War II in a very clever way. They looked at the typical earnings of adjacent *birth cohorts* who were differentially treated by military service.

Their results showed that the simple comparison of veterans of WWII to nonveterans, even after controlling for other measurable characteristics, is misleading because it is typically impossible to control for *ability*, a key unmeasurable trait in humans.

The argument goes something like this.  Imagine that earnings (or log earnings), $Y$, are a linear function of veteran status, $X$, and ability $Z$:

$$ Y_i = \alpha + \beta_0 X_i + \gamma Z_i + \epsilon_i 
$$

We can't typically measure ability, $Z_i$, but we know that it is positively correlated with veteran status and with earnings:  $corr(X_i,Z_i) > 0$ and $corr(Y_i,Z_i) > 0$. Although not true of all drafts, the World War II draft typically drew healthy, higher-ability individuals from the civilian population into service. With this in mind, if we instead estimated the equation without ability:

$$ Y_i = \alpha + \beta_1 X_i + \epsilon_i 
$$

then we would find that our estimate of $\beta_1$ would be biased upward because $Z_i$, which is positively correlated, is an omitted variable.

As we have done recently, let's use the very helpful <a href="http://statsmodels.sourceforge.net/">Statsmodels</a> 
module and some <a href="http://pandas.pydata.org/">Pandas</a> functions to run a multivariate regression. 

In [19]:
import statsmodels.api as sm
import pandas as pd

<h3>Ordinary Least Squares and differences between averages</h3> 

Before we get started, let's revisit some earlier data that we examined in order to reveal a helpful point about OLS estimation when $X$ is a dichotomous or binary 0/1 variable. Recall the HRS smoking dataset we looked at in class 7:

In [30]:
smokeweight = Table.read_table('http://demog.berkeley.edu/~redwards/Courses/LS88/c07_smokeweight.csv')
smokeweight

hhidpn,ragender,r8age,r8weight,r9weight,r8smoken,r9smoken,ones
3010,1,70,71.6672,65.317,0,0,1
3020,2,67,65.317,68.0385,0,0,1
10001010,1,66,72.5744,72.5744,0,0,1
10003030,2,50,58.9667,72.5744,0,0,1
10004010,1,66,102.511,100.697,0,0,1
10004040,2,60,77.1103,74.8423,0,0,1
10013010,1,68,108.862,99.7898,0,0,1
10013040,2,58,64.4098,63.5026,1,1,1
10038010,1,70,74.8423,73.4816,0,0,1
10038040,2,63,64.4098,63.5026,0,0,1


It turns out that to reveal the average weights in wave 8 of smokers and nonsmokers in these data, we can just run OLS on this equation:

$$ r8weight = \alpha + \beta^s \ r8smoken + \epsilon
$$

The estimate of $\alpha$ is the average weight among nonsmokers, for whom $r8smoken$ is zero. And the estimate of $\beta^s$ is the additional average weight among smokers.

Here is how we originally proceeded, by using `.where()`:

In [31]:
#Filter the table; only include rows where r8smoken==1. These are current smokers in wave 8
smoker8 = smokeweight.where('r8smoken',1)
nonsmoker8 = smokeweight.where('r8smoken',0)

<font color="blue">Uncomment the next several lines to reveal these average differences.</font>

In [1]:
#smoker8['r8weight'].mean()

In [2]:
#nonsmoker8['r8weight'].mean()

In [3]:
#smoker8['r8weight'].mean() - nonsmoker8['r8weight'].mean() 

Now let's run OLS instead, after we switch data types:

In [35]:
dfsmokeweight = smokeweight.to_df()
type(dfsmokeweight)

pandas.core.frame.DataFrame

<font color="blue">Uncomment the code below to reveal how OLS also shows us average differences.</font>

In [4]:
# Our x variable is our dichotmous indicator of being a smoker in wage 8
# And our y variable is weight in wave 8
#x = dfsmokeweight[['ones','r8smoken']]
#y = dfsmokeweight['r8weight']
#multiple_regress = sm.OLS(y, x).fit()
#multiple_regress.summary()

<font color = "magenta">Isn't this just kind of awesome?</font> The magic of OLS is that it's all about averages. The $\alpha$ coefficient is the average $Y$ that the "baseline" group gets. When $X$ is dichotomous, then $\beta$ is just the average difference between groups defined by that dichotomous indicator.

<h2>WWII vets</h2>

On to our main activity! Let's look at male veterans of WWII and compare them to nonveterans in the 1980 Census.

These data measure characteristics in 1980 of males born in Q3 or Q4 of 1924, 1926, or 1928. The source is the public-use 5% microsample of the 1980 Census, via <a href="http://usa.ipums.org">IPUMS</a>. 
<ul>
<li> <b>birthyr</b> = year of birth 
<li> <b>birthqtr</b> = quarter of birth 
<li> <b>vetwwii</b> = 0/1 veteran of WWII era 
<li> <b>incwage</b> = wage and salary income in nominal dollars, topcoded at 75,000 
<li> <b>disabwrk1</b> = dichotomous 0/1 measure of whether any disability limits work 
<li> <b>disabwrk2</b> = trichotomous 0/1/2 measure of whether disability doesn't (0), limits (1), or prevents (2) work
<li> <b>ones</b> = vector of 1's, useful for OLS
<li> <b>born192x</b> = dichotomous indicator variables of being born in 1924, 1926, or 1928
</ul>

In [5]:
TableWWII = Table.read_table("http://demog.berkeley.edu/~redwards/Courses/LS88/c10_wwii.csv")
TableWWII

serial,pernum,birthyr,incwage,vetwwii,birthqtr,disabwrk2,disabwrk1,ones,born1924,born1926,born1928
223,1,1924,35005,1,4,0,0,1,1,0,0
272,1,1926,54005,1,3,0,0,1,0,1,0
291,1,1926,18005,1,4,0,0,1,0,1,0
327,1,1928,26005,0,4,0,0,1,0,0,1
422,1,1928,72005,1,4,0,0,1,0,0,1
468,1,1928,45005,0,4,0,0,1,0,0,1
625,1,1928,19005,0,4,0,0,1,0,0,1
694,1,1928,35005,0,3,1,1,1,0,0,1
913,1,1928,12005,0,3,0,0,1,0,0,1
1126,1,1924,0,1,3,2,1,1,1,0,0


Let's run ordinary least squares. As usual, let's switch data structures so we can use pandas. Below, the `.to_df()` method generates a Pandas dataframe containing the same data as the table.

In [7]:
WWII = TableWWII.to_df()
type(WWII)

pandas.core.frame.DataFrame

First let's estimate this model using OLS:

$$ Y_i = \alpha + \beta_1 X_i + \epsilon_i 
$$

In words: we are positing that veteran status ($X$) raises earnings ($Y$) by $\beta_1$. We aren't controlling for other characteristics like ability ($Z$) that are also positively correlated with $Y$, so this estimation is likely to produce an estimate of $\beta_1$ that is biased upward because of omitted variable bias.

In [10]:
# Our x variable is our dichotmous indicator of being a WWII veteran
# And our y variable is wage and salary income
x = WWII[['ones','vetwwii']]
y = WWII['incwage']
# UNCOMMENT and complete the code:
#multiple_regress = ...
#multiple_regress.summary()

Recall that we have estimated this model:

$$ Y_i = \alpha + \beta_1 X_i + \epsilon_i 
$$
&nbsp;

<font color = "blue">Discuss what you see in the OLS results (`multiple_regress.summary()`). Is the R-squared high or low?  Is the $\beta_1$ coefficient statistically significantly different from zero? In words: what does $\alpha$ mean (it is the coefficient on <b>ones</b>); and what does the $\beta_1$ coefficient mean? </font>

<h3>Many groups, not just 2</h3>

Consider the comparison that Small and Rosenbaum (2008) drew.  They compared earnings across three groups, all men born either in quarter 3 or 4, defined by the birth year 1924, 1926, or 1928.

The main motivation is that men born in 1928 during Q3 or Q4 were not yet 17 years old by the end of WWII in August of 1945. There still are some men in this group who stated that they were WWII veterans, which they could have been if they misrepresented their age when enlisting. (And they might be recalling incorrectly, or not understanding the question.) But overall, the proportion of men in this 1928 "half" birth cohort who served in WWII is low relative to the other two groups in the dataset.

First off, is there an easy way to see the average WWII veteran shares in these groups? Yes! We could select subsets with `.where()` or we can run either of these two OLS models:

$$vetwwii = \ \ \ \ \ \ \ \ \ \ \alpha   \ \ \ \ \ \ \ \ \ \ \ \ + \beta^{1926} born1926 + \beta^{1928} born1928 + \epsilon 
$$

$$vetwwii =  \beta^{1924} born1924 + \beta^{1926} born1926 + \beta^{1928} born1928 + \nu 
$$

The difference is that the top regression includes a constant term (via <b>ones</b>) and the bottom does not.  The average WWII veteran share by birth cohort is easier to see when you estimate the bottom regression, without a constant term; you don't have to add the coefficients together, you just look at the single coefficient on the indicator of interest.

Let's do it both ways.  Here's the top equation:

In [11]:
x = WWII[['ones','born1926','born1928']]
y = WWII['vetwwii']
# UNCOMMENT ME:
#multiple_regress = sm.OLS(y, x).fit()
#multiple_regress.summary()

And here's the bottom equation. Note the omission of <b>ones</b>.

In [12]:
x = WWII[['born1924','born1926','born1928']]
y = WWII['vetwwii']
# UNCOMMENT ME:
#multiple_regress = sm.OLS(y, x).fit()
#multiple_regress.summary()

<font color = "blue">State in a sentence or two the shares of these three "half" birth cohorts that are WWII veterans.</font>

As we have seen, there are very large differences in the shares of these birth cohorts who participated in WWII.

Think back to our OLS results from the simple model of earnings as a function of WWII veteran status:

$$Y_i = \alpha + \beta_1 X_i + \epsilon_i
$$
or with variable names:
$$incwage = \alpha + \beta_1 vetwwii + \epsilon
$$



<font color="blue">If the positive OLS estimate of $\beta_1$ that we found were in fact the true causal effect of WWII veteran status on earnings, and given the results we saw immediately above regarding the WWII veteran shares of these birth cohorts, then what would we expect to see if we compared the earnings of the 1924, 1926, and 1928 birth cohorts? Which should have higher earnings?</font>

Now let's test this hypothesis. Run OLS on this model:

$$incwage =  \alpha + \beta^{1926} born1926 + \beta^{1928} born1928 + \epsilon 
$$

This sets up a handy hypothesis test (or two): $\beta^{1926} = \beta^{1928} = 0$, or in words, that the average earnings are the same across each of these half-year birth cohorts. Based on your answer immediately above, we probably have a separate more specific prior about $\beta^{1928} < 0$ or $\beta^{1928} > 0$.

In [13]:
# CONSTRUCT ME:
x = ...
y = ...
# UNCOMMENT ME
#multiple_regress = sm.OLS(y, x).fit()
#multiple_regress.summary()

<font color="blue">Discuss what this hypothesis test reveals. Are you surprised? Is this result consistent with the OLS estimate of $\beta_1$ from before?</font>

<h2>Do health inequalities follow the same story?</h2>

Let's look at health too!  We don't have a ton of health metrics in the 1980 Census, but we do know about disabilities that limit work activity.  Based on the questions in the Census, we have two measures: 
<ul>
<li><b>disabwrk1</b>, a 0/1 dichotomous measure of any disabilities limiting work
<li><b>disabwrk2</b>, a 0/1/2 trichotomous measure of no disabilities (0), disabilities limiting work (1), and disabilities preventing work (2)
</ul>

The story with health in general could be different than with earnings, or it could be the same. High ability typically also correlates with better health. War-related injuries or exposures could reduce health, but we would probably also expect earnings to be hurt as well.  

An additional problem here is that the question asks about health only insofar as it impinges work ability, which implies a rather direct connection between these two measures of health and earnings.

Let's first explore how disability-related work limitation is correlated with earnings. Run this model:

$$incwage = \alpha + \beta_d \ disabwrk1 + \epsilon
$$


In [14]:
x = WWII[['ones','disabwrk1']]
y = WWII['incwage']
# UNCOMMENT ME:
#multiple_regress = sm.OLS(y, x).fit()
#multiple_regress.summary()

<font color="blue">Discuss what you see.</font>

Now let's replicate Small and Rosenbaum's (2008) methodology, only modeling <b>disabrwrk1</b> rather than <b>incwage</b>

Like before, let's first estimate this model using OLS:

$$ disabwrk1 = \alpha + \beta^d_1 vetwwii + \epsilon_i 
$$

In words: we are positing that veteran status ($X$) raises disability ($Y$) by $\beta^d_1$ (or lowers disability if $\beta^d_1 < 0$. Like before, other characteristics like ability ($Z$) are probably also positively correlated with $Y$, so this estimation is likely to produce an estimate of $\beta^d_1$ that is biased upward because of omitted variable bias.

In [16]:
# CONSTRUCT ME:
x = ...
y = ...
# UNCOMMENT ME:
#multiple_regress = sm.OLS(y, x).fit()
#multiple_regress.summary()

<font color="blue">Discuss what you see.</font>

<font color="blue">Given what we saw before about veteran shares in these birth cohorts, what would we expect to see across the three average disability rates within the 1924, 1926, and 1928 birth cohorts, if this $\beta_1^d$ from above were an accurate measure of the causal influence of veteran status on health?.</font>

Now let's test this hypothesis. Run OLS on this model:

$$disabwrk1 =  \alpha + \beta_d^{1926} born1926 + \beta_d^{1928} born1928 + \epsilon 
$$

Like before, this sets up a handy hypothesis test. Comparing these $\beta$'s reveals whether average disability rates are the same across each of these half-year birth cohorts. Based on your answer immediately above, we have a separate more specific prior about $\beta^{1928} < 0$ or $\beta^{1928} > 0$.

In [17]:
# CONSTRUCT ME
x = ...
y = ...
# UNCOMMENT ME:
#multiple_regress = sm.OLS(y, x).fit()
#multiple_regress.summary()

<font color="blue">Discuss what you see.</font>